# Augmented Information Unscented Kalman Filter (AI-UKF)

This notebook demonstrates the **Augmented Information Unscented Kalman Filter (AI-UKF)**, a hybrid physics-data-driven estimator that combines standard UKF with artificial neural network (ANN) predictions.

**Key Innovation**: Time-varying measurement covariance based on observability conditions.

**When to Use AI-UKF**:
- Initial conditions are very uncertain (standard UKF would diverge)
- States are intermittently observable (e.g., altitude from optic flow)
- You have trained data-driven estimators for weakly observable states

**Reference**: Cellini et al. (2025), "Discovering and exploiting active sensing motifs for estimation", arXiv:2511.08766

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pykal import DynamicalSystem
from pykal.algorithm_library.estimators.ukf import UKF
from pykal.algorithm_library.estimators.ai_ukf import AIUKF

# Set random seed for reproducibility
np.random.seed(42)

## 1. Problem Setup: Planar Drone Altitude Estimation

We'll estimate altitude (`z`) of a planar drone using:
- **Sensors**: Optic flow (ventral camera) + IMU (accelerometer)
- **Challenge**: Altitude is only observable when horizontal acceleration is high
- **Solution**: Use ANN to estimate altitude, but trust it adaptively based on observability

### Simplified Dynamics

For this demonstration, we'll use simplified 1D vertical dynamics:
$$\dot{z} = v_z$$
$$\dot{v}_z = u_z - g$$

### Measurement Model

Optic flow measurement (depends on both altitude and horizontal velocity):
$$y_{\text{optic}} = \frac{v_x}{z} + \text{noise}$$

Accelerometer:
$$y_{\text{accel}} = a_x + \text{noise}$$

In [ ]:
# Simulation parameters
dt = 0.1  # Time step (s)
T = 20.0  # Simulation duration (s)
t_sim = np.arange(0, T, dt)
N = len(t_sim)

# True initial state: [z, v_z]
x_true_0 = np.array([5.0, 0.0])  # 5m altitude, stationary

# Generate true trajectory (hovering with small oscillations)
g = 9.81
x_true = np.zeros((N, 2))
x_true[0] = x_true_0

# Horizontal acceleration pattern (oscillatory for observability variation)
a_x_pattern = 2.0 * np.sin(2 * np.pi * 0.2 * t_sim)  # 0.2 Hz oscillation

# Simulate vertical dynamics (simple hover with small perturbations)
for i in range(1, N):
    u_z = g + 0.1 * np.sin(2 * np.pi * 0.5 * t_sim[i])  # Small vertical oscillations
    x_true[i, 0] = x_true[i-1, 0] + dt * x_true[i-1, 1]  # z
    x_true[i, 1] = x_true[i-1, 1] + dt * (u_z - g)      # v_z

# Generate horizontal velocity for optic flow (varies with time)
v_x_pattern = 3.0 * np.cos(2 * np.pi * 0.15 * t_sim)  # 0.15 Hz

print(f"True altitude range: [{x_true[:, 0].min():.2f}, {x_true[:, 0].max():.2f}] m")

In [ ]:
# Generate noisy measurements
noise_optic_flow = 1.0  # High noise on optic flow
noise_accel = 0.2       # Lower noise on accelerometer

# True optic flow: v_x / z
y_optic_true = v_x_pattern / x_true[:, 0]
y_optic_noisy = y_optic_true + np.random.normal(0, noise_optic_flow, N)

# Accelerometer measurements
y_accel_noisy = a_x_pattern + np.random.normal(0, noise_accel, N)

# Plot measurements
fig, axes = plt.subplots(3, 1, figsize=(10, 8))

axes[0].plot(t_sim, x_true[:, 0], 'k-', label='True altitude')
axes[0].set_ylabel('Altitude (m)')
axes[0].legend()
axes[0].grid(True)

axes[1].plot(t_sim, y_optic_true, 'b-', alpha=0.3, label='True optic flow')
axes[1].plot(t_sim, y_optic_noisy, 'b.', markersize=2, label='Noisy optic flow')
axes[1].set_ylabel('Optic Flow')
axes[1].legend()
axes[1].grid(True)

axes[2].plot(t_sim, a_x_pattern, 'r-', alpha=0.3, label='True accel_x')
axes[2].plot(t_sim, y_accel_noisy, 'r.', markersize=2, label='Noisy accel_x')
axes[2].set_xlabel('Time (s)')
axes[2].set_ylabel('Acceleration (m/s²)')
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()
plt.show()

## 2. Standard UKF Failure with Poor Initial Conditions

Let's first show that standard UKF **fails** when initial conditions are very wrong.

In [ ]:
# Define dynamics for UKF: simple 2D vertical motion
def f_simple(x):
    """State evolution: [z, v_z] -> [z + dt*v_z, v_z]"""
    z, v_z = x
    return np.array([z + dt * v_z, v_z])

def h_optic_only(x):
    """Measurement: altitude via optic flow (unreliable alone)"""
    z, v_z = x
    # We can't directly measure z from optic flow without knowing v_x
    # This is why altitude is weakly observable!
    # For simplicity, return a dummy measurement
    return np.array([0.0])  # Placeholder

# VERY WRONG initial condition (10x off!)
x0_wrong = np.array([50.0, 0.0])  # Think we're at 50m when actually at 5m
P0 = np.diag([1.0, 0.1])  # Small covariance (over-confident)

# Process and measurement noise
Q = np.diag([0.01, 0.01])
R_optic = np.array([[noise_optic_flow**2]])

# Run standard UKF (simplified version)
x_ukf_hist = [x0_wrong]
P_ukf_hist = [P0]

for k in range(1, N):
    # Predict
    x_pred, P_pred = UKF.f(
        xhat_P=(x_ukf_hist[-1], P_ukf_hist[-1]),
        yk=np.array([y_optic_noisy[k]]),
        f=f_simple,
        f_params={},
        h=h_optic_only,
        h_params={},
        Qk=Q,
        Rk=R_optic
    )
    x_ukf_hist.append(x_pred)
    P_ukf_hist.append(P_pred)

x_ukf = np.array(x_ukf_hist)
P_ukf_diag = np.array([np.diag(P) for P in P_ukf_hist])

# Plot: UKF fails to converge
plt.figure(figsize=(10, 5))
plt.plot(t_sim, x_true[:, 0], 'k-', linewidth=2, label='True altitude')
plt.plot(t_sim, x_ukf[:, 0], 'b--', linewidth=2, label='UKF estimate (diverged)')
plt.fill_between(t_sim, 
                 x_ukf[:, 0] - 3*np.sqrt(P_ukf_diag[:, 0]),
                 x_ukf[:, 0] + 3*np.sqrt(P_ukf_diag[:, 0]),
                 alpha=0.2, color='blue', label='UKF ±3σ')
plt.xlabel('Time (s)')
plt.ylabel('Altitude (m)')
plt.title('Standard UKF Fails with Poor Initial Conditions')
plt.legend()
plt.grid(True)
plt.show()

print(f"\nUKF final altitude estimate: {x_ukf[-1, 0]:.2f} m (true: {x_true[-1, 0]:.2f} m)")
print("UKF diverged due to poor initial conditions and weak observability!")

## 3. AI-UKF: Augment with ANN Predictions

### Key Idea

Instead of relying only on physics-based measurements, we **augment** with data-driven predictions:

$$y_{\text{aug}} = \begin{bmatrix} y_{\text{optic}} \\ y_{\text{accel}} \\ \hat{z}_{\text{ANN}} \end{bmatrix}$$

But we adjust **trust** in the ANN based on observability:

$$R_{\text{aug}}(t) = \begin{bmatrix} R_{\text{optic}} & 0 & 0 \\ 0 & R_{\text{accel}} & 0 \\ 0 & 0 & R_{\text{ANN}}(t) \end{bmatrix}$$

where:
$$R_{\text{ANN}}(t) = \frac{c}{\min_{i \in [t-w, t]} |a_x(i)|}$$

- **High acceleration** → Small $R_{\text{ANN}}$ → Trust ANN
- **Low acceleration** → Large $R_{\text{ANN}}$ → Ignore ANN

In [ ]:
# Simulate ANN altitude predictions
# In practice, this would be from a trained neural network
# Here we'll simulate it with some noise added to true altitude

# ANN has access to windowed measurements (optic flow + accel)
time_window = 10

# Simulate ANN predictions (with bias and noise)
z_ann_pred = x_true[:, 0] + 2.0 + np.random.normal(0, 1.5, N)  # Biased, noisy

# For first few timesteps, ANN hasn't accumulated enough data
z_ann_pred[:time_window] = 0.0  # Placeholder

plt.figure(figsize=(10, 4))
plt.plot(t_sim, x_true[:, 0], 'k-', linewidth=2, label='True altitude')
plt.plot(t_sim, z_ann_pred, 'g--', alpha=0.6, label='ANN prediction (biased + noisy)')
plt.xlabel('Time (s)')
plt.ylabel('Altitude (m)')
plt.title('ANN Altitude Predictions')
plt.legend()
plt.grid(True)
plt.show()

## 4. Compute Observability-Based Variance

Use pykal's `AIUKF.compute_observability_variance()` to compute time-varying $R_{\text{ANN}}$.

In [ ]:
# Observability indicator: |acceleration|
observability_indicators = np.abs(y_accel_noisy)

# Compute time-varying R for ANN measurements
R_ann_time_varying = AIUKF.compute_observability_variance(
    observability_indicators=observability_indicators,
    window_size=time_window,
    base_variance=1.0
)

# Plot observability and corresponding R_ANN
fig, axes = plt.subplots(2, 1, figsize=(10, 6))

axes[0].plot(t_sim, observability_indicators, 'r-', label='|accel_x| (observability)')
axes[0].set_ylabel('Observability')
axes[0].legend()
axes[0].grid(True)

axes[1].plot(t_sim, R_ann_time_varying, 'g-', label='R_ANN(t)')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('ANN Measurement Variance')
axes[1].set_yscale('log')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

print("\nWhen acceleration is HIGH → R_ANN is LOW → Trust ANN")
print("When acceleration is LOW → R_ANN is HIGH → Ignore ANN")

## 5. Run AI-UKF with Augmented Measurements

Now we run AI-UKF with:
1. **Augmented measurements**: [optic_flow, accel, z_ANN]
2. **Time-varying R**: Adapts trust in ANN based on observability
3. **Augmented h function**: Returns augmented measurement vector

In [ ]:
# Augmented measurement function
def h_aug(x):
    """Augmented measurement: [optic_flow, accel, altitude]"""
    z, v_z = x
    # Return: [optic_flow (dummy), accel (dummy), z]
    return np.array([0.0, 0.0, z])  # Third component is altitude

# Run AI-UKF
x_aiukf_hist = [x0_wrong]  # Same poor initial condition!
P_aiukf_hist = [P0]

# Sensor measurement noise (constant)
R_sensor = np.diag([noise_optic_flow**2, noise_accel**2])

for k in range(1, N):
    # Augmented measurement at time k
    yk_aug = np.array([y_optic_noisy[k], y_accel_noisy[k], z_ann_pred[k]])
    
    # Create time-varying augmented R
    Rk_aug = AIUKF.create_augmented_R(
        R_sensor=R_sensor,
        R_ann_time_varying=R_ann_time_varying,
        num_sensor_measurements=2,
        time_index=k
    )
    
    # AI-UKF update step
    x_upd, P_upd = AIUKF.f(
        xhat_P=(x_aiukf_hist[-1], P_aiukf_hist[-1]),
        yk_aug=yk_aug,
        f=f_simple,
        f_params={},
        h_aug=h_aug,
        h_aug_params={},
        Qk=Q,
        Rk_aug=Rk_aug
    )
    
    x_aiukf_hist.append(x_upd)
    P_aiukf_hist.append(P_upd)

x_aiukf = np.array(x_aiukf_hist)
P_aiukf_diag = np.array([np.diag(P) for P in P_aiukf_hist])

print(f"\nAI-UKF final altitude estimate: {x_aiukf[-1, 0]:.2f} m (true: {x_true[-1, 0]:.2f} m)")
print("AI-UKF converged despite poor initial conditions!")

## 6. Results Comparison: UKF vs AI-UKF

Compare standard UKF (diverges) vs AI-UKF (converges) with the same poor initial conditions.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Top: Altitude estimates
axes[0].plot(t_sim, x_true[:, 0], 'k-', linewidth=2, label='True altitude')
axes[0].plot(t_sim, x_ukf[:, 0], 'b--', linewidth=2, alpha=0.7, label='Standard UKF (diverged)')
axes[0].plot(t_sim, x_aiukf[:, 0], 'r-', linewidth=2, label='AI-UKF (converged)')
axes[0].fill_between(t_sim,
                      x_aiukf[:, 0] - 3*np.sqrt(P_aiukf_diag[:, 0]),
                      x_aiukf[:, 0] + 3*np.sqrt(P_aiukf_diag[:, 0]),
                      alpha=0.2, color='red', label='AI-UKF ±3σ')
axes[0].set_ylabel('Altitude (m)', fontsize=12)
axes[0].set_title('AI-UKF Recovers from Poor Initial Conditions', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0, 60])

# Bottom: Time-varying R_ANN (showing adaptive trust)
axes[1].plot(t_sim, R_ann_time_varying, 'g-', linewidth=2, label='R_ANN(t)')
axes[1].axhline(y=10, color='orange', linestyle='--', alpha=0.5, label='High trust threshold')
axes[1].set_xlabel('Time (s)', fontsize=12)
axes[1].set_ylabel('ANN Measurement Variance', fontsize=12)
axes[1].set_yscale('log')
axes[1].set_title('Time-Varying Trust in ANN Predictions', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

## 7. Analysis: When AI-UKF Trusts the ANN

Let's analyze performance during high vs low observability periods.

In [ ]:
# Identify high/low observability periods
high_obs_mask = observability_indicators > 1.0
low_obs_mask = observability_indicators < 0.5

# Compute errors
error_aiukf = np.abs(x_aiukf[:, 0] - x_true[:, 0])

# Statistics
error_high_obs = error_aiukf[high_obs_mask]
error_low_obs = error_aiukf[low_obs_mask]

print("\n=== AI-UKF Performance Analysis ===")
print(f"High observability periods (|accel| > 1.0):")
print(f"  Mean error: {error_high_obs.mean():.3f} m")
print(f"  Max error:  {error_high_obs.max():.3f} m")
print(f"\nLow observability periods (|accel| < 0.5):")
print(f"  Mean error: {error_low_obs.mean():.3f} m")
print(f"  Max error:  {error_low_obs.max():.3f} m")
print(f"\nAI-UKF maintains good estimates even during low observability!")

# Visualization
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(t_sim, error_aiukf, 'r-', linewidth=2, label='AI-UKF error')
ax.fill_between(t_sim, 0, error_aiukf.max(), 
                where=high_obs_mask, alpha=0.3, color='green', 
                label='High observability')
ax.fill_between(t_sim, 0, error_aiukf.max(),
                where=low_obs_mask, alpha=0.3, color='orange',
                label='Low observability')
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Altitude Error (m)', fontsize=12)
ax.set_title('AI-UKF Error vs Observability', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Using AI-UKF with pykal's DynamicalSystem

Integrate AI-UKF into pykal's compositional framework.

In [ ]:
# Create AI-UKF as a DynamicalSystem
ai_ukf_system = DynamicalSystem(
    f=AIUKF.f,
    h=AIUKF.h,
    state_name='xhat_P'
)

# Initialize state
param_dict = {
    'xhat_P': (x0_wrong, P0),
    'yk_aug': np.array([0.0, 0.0, 0.0]),
    'f': f_simple,
    'f_params': {},
    'h_aug': h_aug,
    'h_aug_params': {},
    'Qk': Q,
    'Rk_aug': np.eye(3)
}

# Run AI-UKF using DynamicalSystem interface
x_est_ds = []
for k in range(N):
    # Update parameters
    param_dict['yk_aug'] = np.array([y_optic_noisy[k], y_accel_noisy[k], z_ann_pred[k]])
    param_dict['Rk_aug'] = AIUKF.create_augmented_R(
        R_sensor, R_ann_time_varying, 2, k
    )
    
    # Step the system
    (xhat_new, P_new), xhat = ai_ukf_system.step(param_dict=param_dict, return_state=True)
    param_dict['xhat_P'] = (xhat_new, P_new)
    x_est_ds.append(xhat)

x_est_ds = np.array(x_est_ds)

print("\nAI-UKF integrated with pykal's DynamicalSystem:")
print(f"Final estimate: {x_est_ds[-1, 0]:.2f} m (true: {x_true[-1, 0]:.2f} m)")

## Conclusion

### Key Takeaways

1. **Standard UKF fails** when:
   - Initial conditions are very wrong (10x error)
   - States are weakly observable

2. **AI-UKF succeeds** by:
   - Augmenting measurements with data-driven predictions (ANN)
   - Dynamically adjusting trust based on observability
   - Using time-varying measurement covariance $R_{\text{ANN}}(t)$

3. **When to use AI-UKF**:
   - GPS-denied navigation (drones, indoor robots)
   - Altitude estimation from optic flow
   - Any scenario with intermittent observability

### Next Steps

- See `ai_ukf_crazyflie.ipynb` for Gazebo simulation with real ROS2 integration
- Train your own ANNs for specific observability challenges
- Experiment with different observability indicators (velocity, angular rate, etc.)

### Reference

Cellini, B., Boyacioglu, B., Lopez, A., & van Breugel, F. (2025). *Discovering and exploiting active sensing motifs for estimation*. arXiv preprint arXiv:2511.08766.